In [263]:
import pandas as pd
import os
from dotenv import load_dotenv, dotenv_values
env = dotenv_values('.env')

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import pyautogui
import requests
from bs4 import BeautifulSoup


import time

# 1. Entrar no SEI

In [267]:
driver = webdriver.Chrome()
driver.get(env['SITE_SEI'])

Exception managing chrome: error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)


# 2. Login

In [269]:
usuario_sei = env['CPF_SEI']
senha_sei = env['SENHA_SEI']

In [271]:
driver.find_element("xpath", '//*[@id="txtUsuario"]').send_keys(usuario_sei)
driver.find_element("xpath", '//*[@id="pwdSenha"]').send_keys(senha_sei)
driver.find_element("xpath", '//*[@id="selOrgao"]').send_keys('CGE')
driver.find_element("xpath", '//*[@id="sbmLogin"]').click()

# SELECIONAR UNIDADE CERTA

unidade_usuario = driver.find_element("xpath", '//*[@id="selInfraUnidades"]')
Select(unidade_usuario).select_by_visible_text(env['UNIDADE'])

# 3. Consulta do Processo

In [272]:
# importando o df com os processos

df = pd.read_excel('lista_processos.xlsx')
df

,Processo Mãe,Data Horário,Unidade Atual,Usuário CPF,Descrição
0,E:35032.0000003108/2024,NaN,NaN,NaN,NaN
1,E:44080.0000000488/2023,NaN,NaN,NaN,NaN
2,E:04799.0000000732/2024,NaN,NaN,NaN,NaN
3,E:01204.0000006582/2024,NaN,NaN,NaN,NaN
4,E:01104.0000000216/2024,NaN,NaN,NaN,NaN
...,...,...,...,...,...
74,E:34000.0000027813/2024,NaN,NaN,NaN,NaN
75,E:02100.0000004489/2024,NaN,NaN,NaN,NaN
76,E:02102.0000005279/2024,NaN,NaN,NaN,NaN
77,E:03300.0000002332/2024,NaN,NaN,NaN,NaN


In [275]:
df.iloc[0 , 0]

'E:35032.0000003108/2024'

In [ ]:
df

In [ ]:
time.sleep(2)
driver.find_element("xpath", '//*[@id="txtPesquisaRapida"]').send_keys(processos)
driver.find_element("xpath", '//*[@id="txtPesquisaRapida"]').send_keys(Keys.ENTER)

# 4. Pegar a unidade atual

In [228]:
# Alternar para o iframe 'ifrArvore'
iframe_arvore = driver.find_element('name', "ifrArvore")
driver.switch_to.frame(iframe_arvore)

In [229]:
driver.find_element("xpath", '//*[@id="divConsultarAndamento"]/a/span').click() # consultar andamento

In [253]:
# Alternar para o iframe 'ifrVisualizacao'
driver.switch_to.default_content()
iframe_visualizacao = driver.find_element('name', "ifrVisualizacao")
driver.switch_to.frame(iframe_visualizacao)

In [257]:
data_hora = driver.find_element('xpath', '//*[@id="tblHistorico"]/tbody/tr[2]/td[1]').text  # XPath para 1ª linha, coluna 1 (data hora)
unidade = driver.find_element('xpath', '//*[@id="tblHistorico"]/tbody/tr[2]/td[2]').text  # XPath para 1ª linha, coluna 2 (Unidade)
usuario = driver.find_element('xpath', '//*[@id="tblHistorico"]/tbody/tr[2]/td[3]').text  # XPath para 1ª linha, coluna 3 (usuario)
descricao = driver.find_element('xpath', '//*[@id="tblHistorico"]/tbody/tr[2]/td[4]').text  # XPath para 1ª linha, coluna 4 (descricao)
print(usuario)

13293502431


In [161]:
# Voltar ao contexto principal
driver.switch_to.default_content()

# Looping

In [262]:
df.columns

Index(['Processo Mãe', 'Data Horário', 'Unidade Atual', 'Usuário CPF',
       'Descrição'],
      dtype='object')

In [273]:
for processo in df['Processo Mãe']:
    time.sleep(2)
    driver.find_element("xpath", '//*[@id="txtPesquisaRapida"]').send_keys(processo)
    driver.find_element("xpath", '//*[@id="txtPesquisaRapida"]').send_keys(Keys.ENTER)

    # Alternar para o iframe 'ifrArvore'
    iframe_arvore = driver.find_element('name', "ifrArvore")
    driver.switch_to.frame(iframe_arvore)

    driver.find_element("xpath", '//*[@id="divConsultarAndamento"]/a/span').click() # consultar andamento
        
    # Alternar para o iframe 'ifrVisualizacao'
    driver.switch_to.default_content()
    iframe_visualizacao = driver.find_element('name', "ifrVisualizacao")
    driver.switch_to.frame(iframe_visualizacao)

    # Extrai os dados da primeira linha da tabela
    data_hora = driver.find_element(By.XPATH, '//*[@id="tblHistorico"]/tbody/tr[2]/td[1]').text  # Coluna 1: Data/Horário
    unidade = driver.find_element(By.XPATH, '//*[@id="tblHistorico"]/tbody/tr[2]/td[2]').text    # Coluna 2: Unidade
    usuario = driver.find_element(By.XPATH, '//*[@id="tblHistorico"]/tbody/tr[2]/td[3]').text    # Coluna 3: Usuário
    descricao = driver.find_element(By.XPATH, '//*[@id="tblHistorico"]/tbody/tr[2]/td[4]').text  # Coluna 4: Descrição

    # Cria um dicionário para organizar os dados extraídos
    dados = {
        "Data Horário": data_hora,
        "Unidade Atual": unidade,
        "Usuário CPF": usuario,
        "Descrição": descricao
    }

    # Atualiza o DataFrame de forma mais eficiente
    df.loc[df['Processo Mãe'] == processo, ['Data Horário', 'Unidade Atual', 'Usuário CPF', 'Descrição']] = [
        dados["Data Horário"], dados["Unidade Atual"], dados["Usuário CPF"], dados["Descrição"]
    ]

    # Voltar ao contexto principal
    driver.switch_to.default_content()

C:\Users\vinicius.ventura\AppData\Local\Temp\ipykernel_15012\115940481.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '31/10/2024 08:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['Processo Mãe'] == processo, ['Data Horário', 'Unidade Atual', 'Usuário CPF', 'Descrição']] = [
C:\Users\vinicius.ventura\AppData\Local\Temp\ipykernel_15012\115940481.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'CGE PDTR' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['Processo Mãe'] == processo, ['Data Horário', 'Unidade Atual', 'Usuário CPF', 'Descrição']] = [
C:\Users\vinicius.ventura\AppData\Local\Temp\ipykernel_15012\115940481.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an e

KeyboardInterrupt: 

In [274]:
df

,Processo Mãe,Data Horário,Unidade Atual,Usuário CPF,Descrição
0,E:35032.0000003108/2024,31/10/2024 08:00,CGE PDTR,12640788469,Processo recebido na unidade
1,E:44080.0000000488/2023,06/12/2024 08:33,CGE PDTR,13293502431,Conclusão do processo na unidade
2,E:04799.0000000732/2024,05/12/2024 11:31,CGE PDTR,10343116464,Conclusão do processo na unidade
3,E:01204.0000006582/2024,06/12/2024 10:29,CGE PDTR,13293502431,Conclusão do processo na unidade
4,E:01104.0000000216/2024,05/12/2024 10:12,CGE PDTR,13293502431,Conclusão do processo na unidade
...,...,...,...,...,...
74,E:34000.0000027813/2024,NaN,NaN,NaN,NaN
75,E:02100.0000004489/2024,NaN,NaN,NaN,NaN
76,E:02102.0000005279/2024,NaN,NaN,NaN,NaN
77,E:03300.0000002332/2024,NaN,NaN,NaN,NaN
